In [40]:
import os
import numpy as np
from pathlib import Path
import json

In [13]:
# all files which end in .rel in this folder
label_dir='/home/max/data/artifacts/i2b2/2010/hd_data/'

def load_files(folder):
  return [os.path.join(label_dir, folder, f) for f in os.listdir(os.path.join(label_dir, folder)) if os.path.isfile(os.path.join(label_dir, folder, f))]
files = load_files('random')
print(len(files))
files += load_files('test')
print(len(files))

1176
1653


In [17]:
class Concept:
  def __init__(self, line):
    span_info, type_info, attr_info = line.strip().split('||')
    fields = span_info.split(' ')
    start, end = fields[-2], fields[-1]
    start = start.split(':')
    end = end.split(':')
    assert start[0] == end[0], print(span_info)
    self.sentence_idx = int(start[0]) - 1
    self.start = int(start[1])
    self.end = int(end[1])
    self.type = type_info.strip()[3:-1]
    self.assertion = attr_info.strip()[3:-1]

  def __eq__(self, other):
    return self.sentence_idx == other.sentence_idx and \
           self.start == other.start and \
           self.end == other.end and \
           self.type == other.type and \
           self.assertion == other.assertion

  def __hash__(self):
    return hash((self.sentence_idx, self.start, self.end, self.type, self.assertion))

  def __str__(self):
    return f"[sid: {self.sentence_idx}, ({self.start}, {self.end}), type={self.type}, assertion={self.assertion}]"

In [38]:
def get_concepts(txtfile):
  txtfile = Path(txtfile)
  name = txtfile.name.replace('.txt', '')
  confile = os.path.join(label_dir, 'all', (name + '.con'))
  astfile = os.path.join(label_dir, 'all', (name + '.ast'))
  relfile = os.path.join(label_dir, 'all', (name + '.rel'))
  if not os.path.exists(confile):
    return []
  problems = [Concept(line) for line in open(astfile)]
  other_concepts = [Concept(line.strip() + '||a="present"') for line in open(confile)]
  concepts = problems + [c for c in other_concepts if c.type != "problem"]
  sentences = [[w for w in line.strip().split(' ') if len(w.strip()) > 0]
                 for s, line in enumerate(open(str(txtfile)))]
  concept_text = []
  for concept in concepts:
    txt = ' '.join(sentences[concept.sentence_idx][concept.start:concept.end+1])
    concept_text.append(txt)
  return concept_text

ctxt2id = {}
for file in files:
  for ctxt in get_concepts(file):
    if ctxt not in ctxt2id:
      ctxt2id[ctxt] = len(ctxt2id)
print(len(ctxt2id))

29238


In [42]:
with open(os.path.join(label_dir, 'ctxt2id'), 'w') as f:
  json.dump(ctxt2id, f)